<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайны-лес" data-toc-modified-id="Случайны-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайны лес</a></span></li><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Линейная регрессия</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайны-лес" data-toc-modified-id="Случайны-лес-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Случайны лес</a></span></li><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Линейная регрессия</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Импортируем необходимые библиотеки в следующей ячейке:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score 
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import warnings
warnings.simplefilter("ignore")

In [2]:
#КОД РЕВЬЮЕРА
np.random.seed(12345)

Прочитаем файл:

In [3]:
df = pd.read_csv('/datasets/Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Имеются пропуски в столбце Tenure, данные столбец вряд ли заполняется вручную, так как в него должны попадать значения промежутка с даты регистрации клиента в банке по дату формирования списка, поэтому можно предположить, что у клиентов с пустыми значениями дата регистрации была до начала ведения этой базы данных, поэтому стартовая дата не попала в алгоритм расчета, как следствие расчет выдал NaN, с другой стороны NaN значения могли попасть в столбец, если клиент появился в банке меньше года назад. В данном случае определить, что из этого верно не представляется возможным, поэтому заполним данные генератором случайных чисел.

In [4]:
tenure_random = pd.Series(np.random.randint(1, 10, 10000))
df['Tenure'] = df['Tenure'].fillna(tenure_random)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Далее нужно убрать столбцы, которые не помогут нам в машинном обучении:

In [5]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


В результате убраны столбцы: RowNumber, CustomerId, Surname. Все 3 являются идентификаторами, которые могут негативно сказаться на обучении модели, ведь они не влияют на то уйдет клиент или нет.

Далее стандартизируем название столбцов:

In [6]:
df = df.rename(columns = {'CreditScore': 'Credit_Score', 'NumOfProducts': 'Num_Of_Products', 'HasCrCard': 'Has_Cr_Card', 'IsActiveMember': 'Is_Active_Member', 'EstimatedSalary': 'Estimated_Salary'})
df.columns = df.columns.str.lower() 
df

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Далее заменим значения в столбцах geography и gender методом OHE:

In [7]:
df = pd.get_dummies(df, drop_first=True)
df

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Далее разделим датасет на выборки:

In [8]:
target = df['exited']
features = df.drop('exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки:', len(features_train)/len(df)*100, len(target_train)/len(df)*100)
print('Размер валидационной выборки:', len(features_valid)/len(df)*100, len(target_valid)/len(df)*100)

Размер тренировочной выборки: 75.0 75.0
Размер валидационной выборки: 25.0 25.0


Для подбора параметров будем использовать методы кроссвалидации, поэтому созданы только тренировочные и тестовые выборки. Далее масштабируем признаки:

In [9]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

**Вывод** В данных:
* заменены значения NaN;
* стандартизированы заголовки столбцов;
* убраны ненужные для машинного обучения столбцы;
* применен метод OHE;
* датасет разделен на выборки;
* произведено масштабирование.

Теперь выборки готовы для обучения моделей.

## Исследование задачи

Создадим несколько моделей, подберем параметры и по результатам выберем лучшую из них. 

### Решающее дерево

Найдем лучший параметр:

In [10]:
parametrs = {'max_depth': range (1,70)}
model_dtc = DecisionTreeClassifier(random_state=12345)
grid_dtc = GridSearchCV(model_dtc, parametrs, cv=5, scoring='f1')
grid_dtc.fit(features_train, target_train)
grid_dtc.best_params_

{'max_depth': 9}

In [11]:
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=9)
model_dtc.fit(features_train, target_train)
predictions_valid_dtc = model_dtc.predict(features_valid)
probabilities_valid = model_dtc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_dtc))

auc_roc: 0.8002292454400609 f1: 0.5475372279495991


### Случайны лес

Находим лучшие параметры:

In [12]:
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2)}
model_dfc = RandomForestClassifier(random_state=12345)
grid_dfc = GridSearchCV(model_dfc, parametrs, cv=5, scoring='f1')
grid_dfc.fit(features_train, target_train)
grid_dfc.best_params_

{'max_depth': 11, 'n_estimators': 50}

In [13]:
model_dfc = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=11,)
model_dfc.fit(features_train, target_train)
predictions_valid_dfc = model_dfc.predict(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_dfc))

auc_roc: 0.8002292454400609 f1: 0.5470085470085471


### Линейная регрессия

Проверим подходит ли эта модель:

In [14]:
model_log_r = LogisticRegression(random_state=12345, solver='liblinear')
model_log_r.fit(features_train, target_train) 
predictions_valid_log_r = model_log_r.predict(features_valid)
probabilities_valid = model_log_r.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_log_r))

auc_roc: 0.7587995529238305 f1: 0.2948539638386648


Проверим соотномешние параметров в столбце exited:

In [15]:
target.value_counts()/len(target)

0    0.7963
1    0.2037
Name: exited, dtype: float64

**Вывод** В результате получили низкие показатели, судя по всему проблема в том что соотношение отрицательных и положительных ответов в прогнозируемом признаке датасета равно 1:4, попробуем улучшить баланс модели, а затем снова создадим модель логистической регрессии и другие модели на измененных данных.

## Борьба с дисбалансом

Для улучшения баланса сделаем upsample и downsample датасета:

In [16]:
#def upsample(features, target, repeat):
#    features_zeros = features[target == 0]
#    features_ones = features[target == 1]
#    target_zeros = target[target == 0]
#    target_ones = target[target == 1]
#    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
#    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)    
#    features_upsampled, target_upsampled = shuffle(
#        features_upsampled, target_upsampled, random_state=12345)
#    
#    return features_upsampled, target_upsampled
#
#features_upsampled, target_upsampled = upsample(features_train, target_train, 10)


def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.20)

**Вывод** В результате перебора параметров с последующей проверкой в моделях тренировочная выборка будет более сбалансирована, для лучшего баланса в дальнейшем укажем параметр: class_weight='balanced'

## Тестирование модели

### Решающее дерево

Найдем лучший параметр:

In [17]:
parametrs = {'max_depth': range (1,70)}
model_dtc = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
grid_dtc = GridSearchCV(model_dtc, parametrs, cv=5, scoring='f1')
grid_dtc.fit(features_downsampled, target_downsampled)
grid_dtc.best_params_

{'max_depth': 7}

In [18]:
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=7, class_weight='balanced')
model_dtc.fit(features_downsampled, target_downsampled)
predictions_valid_dtc = model_dtc.predict(features_valid)
probabilities_valid = model_dtc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_dtc))

auc_roc: 0.8108287555587262 f1: 0.5739130434782609


### Случайны лес

Находим лучшие параметры:

In [1]:
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2)}
model_dfc = RandomForestClassifier(random_state=12345, class_weight='balanced')
grid_dfc = GridSearchCV(model_dfc, parametrs, cv=5, scoring='f1')
grid_dfc.fit(features_downsampled, target_downsampled)
grid_dfc.best_params_

NameError: name 'RandomForestClassifier' is not defined

In [20]:
model_dfc = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=9, class_weight='balanced')
model_dfc.fit(features_downsampled, target_downsampled)
predictions_valid_dfc = model_dfc.predict(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_dfc))

auc_roc: 0.8108287555587262 f1: 0.6071428571428571


### Линейная регрессия

Проверим подходит ли эта модель:

In [21]:
model_log_r = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_log_r.fit(features_downsampled, target_downsampled) 
predictions_valid_log_r = model_log_r.predict(features_valid)
probabilities_valid = model_log_r.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc:', auc_roc, 'f1:',f1_score(target_valid, predictions_valid_log_r))

auc_roc: 0.7639390264202992 f1: 0.5110073382254836


**Вывод** По условиям задачи нужно выбрать модель с лучшим показателем F1 выше 0.59, ей стала модель случайный лес, её показатель AUC-ROC так же высок, и он не находится на критической отметке в 0.5 равной случайной модели и не противоречит условию поставленной в проекте задачи.